reference: https://github.com/hollance/BlazeFace-PyTorch/blob/master/Anchors.ipynb

In [12]:
import numpy as np

In [13]:
anchor_options = {
    "num_layers": 4,
    "min_scale": 0.1484375, # default: 0.1484375, custom: 0.05
    "max_scale": 0.75,
    "input_size_height": 128,
    "input_size_width": 128,
    "anchor_offset_x": 0.5,
    "anchor_offset_y": 0.5,
    "strides": [8, 16, 16, 16],
    "aspect_ratios": [1.0],
    "reduce_boxes_in_lowest_layer": False,
    "interpolated_scale_aspect_ratio": 1.0,
    "fixed_anchor_size": False,
}

In [14]:
# SSD: Single Shot Multibox Detector: 
#   Choosing scales and aspect ratios for default boxes 수식 (4) 참고
def calculate_scale(min_scale, max_scale, stride_index, num_strides):
    return min_scale + (max_scale - min_scale) * stride_index / (num_strides - 1.0)


def generate_anchors(options):
    strides_size = len(options["strides"])
    assert options["num_layers"] == strides_size

    anchors = []
    layer_id = 0
    while layer_id < strides_size:
        anchor_height = []
        anchor_width = []
        aspect_ratios = []
        scales = []

        # For same strides, we merge the anchors in the same order.
        last_same_stride_layer = layer_id
        # size 8에서 stride 16으로 2번 append, (128 / 16) ** 2 * 6 = 512
        # size 16에서 stride 8로 6번 append, (128 / 8) ** 2 * 2 = 384
        while (last_same_stride_layer < strides_size) and \
              (options["strides"][last_same_stride_layer] == options["strides"][layer_id]):

            # 현재 layer에 해당하는 scale을 계산한다.
            scale = calculate_scale(options["min_scale"],
                                    options["max_scale"],
                                    last_same_stride_layer,
                                    strides_size)

            if last_same_stride_layer == 0 and options["reduce_boxes_in_lowest_layer"]:
                # For first layer, it can be specified to use predefined anchors.
                # 첫 번째 layer에서 사전에 정의된 anchors가 필요한 경우에 다음 if문 내의 코드를 사용할 수 있다.
                aspect_ratios.append(1.0)
                aspect_ratios.append(2.0)
                aspect_ratios.append(0.5)
                scales.append(0.1)
                scales.append(scale)
                scales.append(scale)                
            else:
                for aspect_ratio in options["aspect_ratios"]: 
                    # aspect ratio는 1로 고정. 정사각형 box에 대한 얼굴을 찾을 것이다.
                    aspect_ratios.append(aspect_ratio)
                    scales.append(scale)

                if options["interpolated_scale_aspect_ratio"] > 0.0:
                    # 마지막 layer인 경우, max_scale과 1 사이의 중간값을 scales에 추가한다.
                    scale_next = 1.0 if last_same_stride_layer == strides_size - 1 \
                                     else calculate_scale(options["min_scale"],
                                                          options["max_scale"],
                                                          last_same_stride_layer + 1,
                                                          strides_size)
                    scales.append(np.sqrt(scale * scale_next))
                    aspect_ratios.append(options["interpolated_scale_aspect_ratio"])

            last_same_stride_layer += 1
        
        for i, aspect_ratio in enumerate(aspect_ratios):
            ratio_sqrts = np.sqrt(aspect_ratio)
            anchor_height.append(scales[i] / ratio_sqrts)
            anchor_width.append(scales[i] * ratio_sqrts)            
        
        stride = options["strides"][layer_id]
        feature_map_height = int(np.ceil(options["input_size_height"] / stride)) # np.ceil: 올림함수
        feature_map_width = int(np.ceil(options["input_size_width"] / stride))
        
        for y in range(feature_map_height):
            for x in range(feature_map_width):
                for anchor_id in range(len(anchor_height)):
                    x_center = (x + options["anchor_offset_x"]) / feature_map_width
                    y_center = (y + options["anchor_offset_y"]) / feature_map_height

                    new_anchor = [x_center, y_center, 0, 0]
                    if options["fixed_anchor_size"]:
                        # BlazeFace에선 고정된 aspect ratio를 사용하기 때문에 width, height는 항상 1.0이다.
                        new_anchor[2] = 1.0
                        new_anchor[3] = 1.0
                    else:
                        new_anchor[2] = anchor_width[anchor_id]
                        new_anchor[3] = anchor_height[anchor_id]
                    anchors.append(new_anchor)

        layer_id = last_same_stride_layer

    return anchors

In [15]:
anchors = generate_anchors(anchor_options)

assert len(anchors) == 896

In [16]:
anchors[:50]

[[0.03125, 0.03125, 0.1484375, 0.1484375],
 [0.03125, 0.03125, 0.22759284392125925, 0.22759284392125925],
 [0.09375, 0.03125, 0.1484375, 0.1484375],
 [0.09375, 0.03125, 0.22759284392125925, 0.22759284392125925],
 [0.15625, 0.03125, 0.1484375, 0.1484375],
 [0.15625, 0.03125, 0.22759284392125925, 0.22759284392125925],
 [0.21875, 0.03125, 0.1484375, 0.1484375],
 [0.21875, 0.03125, 0.22759284392125925, 0.22759284392125925],
 [0.28125, 0.03125, 0.1484375, 0.1484375],
 [0.28125, 0.03125, 0.22759284392125925, 0.22759284392125925],
 [0.34375, 0.03125, 0.1484375, 0.1484375],
 [0.34375, 0.03125, 0.22759284392125925, 0.22759284392125925],
 [0.40625, 0.03125, 0.1484375, 0.1484375],
 [0.40625, 0.03125, 0.22759284392125925, 0.22759284392125925],
 [0.46875, 0.03125, 0.1484375, 0.1484375],
 [0.46875, 0.03125, 0.22759284392125925, 0.22759284392125925],
 [0.53125, 0.03125, 0.1484375, 0.1484375],
 [0.53125, 0.03125, 0.22759284392125925, 0.22759284392125925],
 [0.59375, 0.03125, 0.1484375, 0.1484375],
 [0

In [17]:
np.save("anchors.npy", anchors)